<a href="https://colab.research.google.com/github/cowsilver57/sessac_test/blob/main/Chapter6_%EA%B8%B0%EC%B6%9C2%ED%9A%8C_2%EC%9C%A0%ED%98%95(20%EC%9D%BC%EC%B0%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
#github와 colab 연동
!git clone https://github.com/AnalyticsKnight/yemoonsaBigdata

fatal: destination path 'yemoonsaBigdata' already exists and is not an empty directory.


#문제

다음은 E-commerce Shipping 데이터 세트이다.

주어진 훈련 데이터 세트를 활용하여 고객이 주문한 물품의 정시 도착 여부를 예측하고 해당 예측 결과를 csv 파일로 제출하시오.

(결과 제출 양식: 제출한 예측값의 ROC_AUC 점수 결과를 통해 영역별 배점에 따라 최종 점수가 반영될 예정)

#풀이

##데이터 불러오기

In [35]:
import pandas as pd
import numpy as np

X_test = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/204_x_test.csv')
X_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/204_x_train.csv')
y_train = pd.read_csv('/content/yemoonsaBigdata/datasets/Part3/204_y_train.csv')

## 데이터 살펴보기

In [36]:
print(X_train.head())

      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   9902               F             Ship                    3   
1   9501               A             Ship                    4   
2   6111               A             Ship                    4   
3    588               F             Ship                    3   
4  10806               F             Road                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                5                  214                2             medium   
1                2                  201                3             medium   
2                2                  264                3                low   
3                5                  194                2             medium   
4                5                  269                5             medium   

  Gender  Discount_offered  Weight_in_gms  
0      F                 6           4578  
1      F                 4           461

In [37]:
print(y_train.head())

      ID  Reached.on.Time_Y.N
0   9902                    0
1   9501                    0
2   6111                    1
3    588                    1
4  10806                    0


In [38]:
print(X_test.head())

      ID Warehouse_block Mode_of_Shipment  Customer_care_calls  \
0   7007               C             Ship                    4   
1   9793               D             Ship                    4   
2   6593               C           Flight                    7   
3  10527               A             Road                    4   
4   6914               F             Ship                    5   

   Customer_rating  Cost_of_the_Product  Prior_purchases Product_importance  \
0                1                  238                3               high   
1                2                  245                3             medium   
2                4                  272                4                low   
3                1                  204                6                low   
4                5                  256                5             medium   

  Gender  Discount_offered  Weight_in_gms  
0      F                 4           4910  
1      M                 1           473

##데이터 결측치 확인하기

In [39]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [40]:
y_train.isnull().sum()

ID                     0
Reached.on.Time_Y.N    0
dtype: int64

In [41]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

##데이터 전처리

In [42]:
print(X_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6599 non-null   int64 
 1   Warehouse_block      6599 non-null   object
 2   Mode_of_Shipment     6599 non-null   object
 3   Customer_care_calls  6599 non-null   int64 
 4   Customer_rating      6599 non-null   int64 
 5   Cost_of_the_Product  6599 non-null   int64 
 6   Prior_purchases      6599 non-null   int64 
 7   Product_importance   6599 non-null   object
 8   Gender               6599 non-null   object
 9   Discount_offered     6599 non-null   int64 
 10  Weight_in_gms        6599 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 567.2+ KB
None


In [43]:
print(y_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6599 entries, 0 to 6598
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   ID                   6599 non-null   int64
 1   Reached.on.Time_Y.N  6599 non-null   int64
dtypes: int64(2)
memory usage: 103.2 KB
None


In [44]:
print(X_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4400 entries, 0 to 4399
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   4400 non-null   int64 
 1   Warehouse_block      4400 non-null   object
 2   Mode_of_Shipment     4400 non-null   object
 3   Customer_care_calls  4400 non-null   int64 
 4   Customer_rating      4400 non-null   int64 
 5   Cost_of_the_Product  4400 non-null   int64 
 6   Prior_purchases      4400 non-null   int64 
 7   Product_importance   4400 non-null   object
 8   Gender               4400 non-null   object
 9   Discount_offered     4400 non-null   int64 
 10  Weight_in_gms        4400 non-null   int64 
dtypes: int64(7), object(4)
memory usage: 378.2+ KB
None


## 데이터 분류하기

In [45]:
COL_DEL = ['ID']
COL_NUM = ['Customer_care_calls','Customer_rating','Cost_of_the_Product','Prior_purchases','Discount_offered','Weight_in_gms']
COL_CAT = ['Warehouse_block','Mode_of_Shipment','Product_importance','Gender']
COL_Y = ['Reached.on.Time_Y.N']

##데이터 분할하기

In [50]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train[COL_NUM + COL_CAT],
                                            y_train[COL_Y].values.ravel(),
                                            test_size=0.3,
                                            stratify = y_train[COL_Y].values.ravel())

In [51]:
print(y_tr)

[1 0 0 ... 0 1 0]


## 데이터 스케일링

In [54]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[COL_NUM])

X_tr[COL_NUM] = scaler.transform(X_tr[COL_NUM])
X_val[COL_NUM] = scaler.transform(X_val[COL_NUM])
X_test[COL_NUM] = scaler.transform(X_test[COL_NUM])

In [55]:
print(X_tr[COL_NUM].head())

      Customer_care_calls  Customer_rating  Cost_of_the_Product  \
5125            -1.810351        -1.398534             1.114517   
1761            -0.054548         1.411004            -1.392230   
6470             0.823353         0.006235             1.299442   
274             -0.932450         0.708619             0.744669   
2245            -0.932450        -0.696150            -0.837458   

      Prior_purchases  Discount_offered  Weight_in_gms  
5125        -0.363981         -0.200754      -0.148414  
1761        -1.020967         -0.701482       0.502222  
6470         0.293005         -0.764073       0.898234  
274         -0.363981          1.051064       0.059079  
2245        -1.020967          1.051064      -1.195674  


##데이터 인코딩

In [57]:
from sklearn.preprocessing import LabelEncoder

X = pd.concat([X_train[COL_CAT], X_test[COL_CAT]])

for col in COL_CAT:
    le = LabelEncoder()
    le.fit(X[col])
    X_tr[col] = le.transform(X_tr[col])
    X_val[col] = le.transform(X_val[col])
    X_test[col] = le.transform(X_test[col])

    #각 변수의 클래스 확인
    print(col)
    print(le.classes_)
    print('\n')

Warehouse_block
['A' 'B' 'C' 'D' 'F']


Mode_of_Shipment
['Flight' 'Road' 'Ship']


Product_importance
['high' 'low' 'medium']


Gender
['F' 'M']




In [58]:
print(X_tr[COL_CAT].head())

      Warehouse_block  Mode_of_Shipment  Product_importance  Gender
5125                4                 2                   2       1
1761                3                 1                   1       1
6470                2                 2                   2       1
274                 4                 2                   2       1
2245                4                 2                   0       1


##모델 학습시키기

In [60]:
#1. 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
modelRF = RandomForestClassifier(random_state=123)
modelRF.fit(X_tr, y_tr)

RandomForestClassifier(random_state=123)

In [63]:
#2. XGBoost
from xgboost import XGBClassifier
modelXGB = XGBClassifier(random_state=123)
modelXGB.fit(X_tr, y_tr)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=123, ...)

## 모델 평가하기

In [68]:
y_val_predRF = modelRF.predict_proba(X_val)[:,1]
y_val_predXGB = modelXGB.predict_proba(X_val)[:,1]

## 평가 지표 구하기

In [70]:
from sklearn.metrics import roc_auc_score

scoreRF = roc_auc_score(y_val, y_val_predRF)
scoreXGB = roc_auc_score(y_val, y_val_predXGB)

print('Random Forest:', scoreRF)
print('XGBoost:', scoreXGB)

#XGBoost로 모델 설정

Random Forest: 0.7391167215839939
XGBoost: 0.7427610905436179


## 학습, 검증 성능 확인 함수

In [73]:
def get_scores(model, X_tr, X_val, y_tr, y_val):

    y_tr_pred = model.predict_proba(X_tr)[:,1]
    y_val_pred = model.predict_proba(X_val)[:,1]
    tr_score = roc_auc_score(y_tr, y_tr_pred)
    val_score = roc_auc_score(y_val, y_val_pred)

    return f'train: {round(tr_score,4)}, valid: {round(val_score,4)}'

In [78]:
print('Random Forest:\n', get_scores(modelRF, X_tr, X_val, y_tr, y_val),'\n')
print('XGBoost:\n', get_scores(modelXGB, X_tr, X_val, y_tr, y_val))

#Random Forest는 과대적합이 심하게 되어있다.
#따라서 원래대로 XGBoost 모델 사용

Random Forest:
 train: 1.0, valid: 0.7391 

XGBoost:
 train: 0.9954, valid: 0.7428


## 하이퍼 파리미터 조정

In [80]:
modelXGB2 = XGBClassifier(n_estimators=30, max_depth=3, min_child_weight=1, random_state=123)
modelXGB2.fit(X_tr, y_tr)

modelXGB3 = XGBClassifier(n_estimators=30, max_depth=5, min_child_weight=1, random_state=123)
modelXGB3.fit(X_tr, y_tr)

modelXGB4 = XGBClassifier(n_estimators=50, max_depth=3, min_child_weight=1, random_state=123)
modelXGB4.fit(X_tr, y_tr)

modelXGB5 = XGBClassifier(n_estimators=50, max_depth=5, min_child_weight=1, random_state=123)
modelXGB5.fit(X_tr, y_tr)

print("XGBoost2:\n", get_scores(modelXGB2, X_tr, X_val, y_tr, y_val))
print("XGBoost3:\n", get_scores(modelXGB3, X_tr, X_val, y_tr, y_val))
print("XGBoost4:\n", get_scores(modelXGB4, X_tr, X_val, y_tr, y_val))
print("XGBoost5:\n", get_scores(modelXGB5, X_tr, X_val, y_tr, y_val))

#XGBoost 2 선택

XGBoost2:
 train: 0.8015, valid: 0.7501
XGBoost3:
 train: 0.8729, valid: 0.7481
XGBoost4:
 train: 0.8319, valid: 0.7474
XGBoost5:
 train: 0.9252, valid: 0.7469


## 최종 모델로 예측값 생성

In [83]:
pred = modelXGB2.predict_proba(X_test[COL_NUM+COL_CAT])[:,1]
result = pd.DataFrame({'ID':X_test.ID, 'pred':pred})
print(result.head())

      ID      pred
0   7007  0.999097
1   9793  0.999240
2   6593  0.999097
3  10527  0.999097
4   6914  0.999240
